![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Named Entity Recognition using rules

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.4.0
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.0 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

# How the ContextualParser Works

This annotator is a kind of RegexMatcher based on a JSON file, that is defined through the parameter `setJsonPath()`

In this JSON file, you define the regex that you want to match along with the information that will output on metadata field.

For example here, you define the name of an entity that will categorize the matches, the regex value and the  `matchScope` that will tell the regex whether to make a full match or a partial match

```
{
  "entity": "Stage",
  "ruleScope": "sentence",
  "regex": "[cpyrau]?[T][0-9X?][a-z^cpyrau]*",
  "matchScope": "token"
}
```

Ignore the `ruleScope` for the moment, it's always at a `sentence` level. Which means find match on each sentence. So, for example for this text:

  A patient has liver metastases pT1bN0M0 and the T5 primary site may be colon or lung. If the primary site is not clearly identified , this case is cT4bcN2M1, Stage 
  Grouping 88. N4 A child T?N3M1  has soft tissue aM3 sarcoma and the staging has been left unstaged. Both clinical and pathologic staging would be coded pT1bN0M0 as 
  unstageable cT3cN2.Medications started.

val expectedResult = Array("pT1bN0M0", "T5", "cT4bcN2M1", "T?N3M1", "pT1bN0M0", "cT3cN2.Medications")
      val expectedMetadata =
      Array(Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"),
            Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"),
            Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "1"),
            Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "2"),
            Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "3"),
            Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "3")
     )

Whereas, using a `matchScope` at sub-token level it will output:

val expectedResult = Array("pT1b", "T5", "cT4bc", "T?", "pT1b", "cT3c")
    val expectedMetadata =
    Array(Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"),
    Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"),
    Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "1"),
    Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "2"),
    Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "3"),
    Map("field" -> "Stage", "normalized" -> "", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "3")
    )

The `confidence` value is another feature, which is computed  basically using a heuristic approach based on how many matches it has.

To clarify how many matches, this is an example of the JSON file with additional fields that will define the match we want to get

```
{
  "entity": "Gender",
  "ruleScope": "sentence",
  "matchScope": "token",
  "prefix": ["birth", "growing", "assessment"],
  "suffix": ["faster", "velocities"],
  "contextLength": 50,
  "context": ["typical", "grows"]
}
```

for example, prefix and suffix refer to the words that are required to be near the word we want to match.

This two work also with `contextLength` that will tell the maximum distance that prefix or suffix words can be away from the word to match, whereas `context` are words that must be immediately after or before the word to match

Now, there is another feature that can be used. The `dictionary` parameter. In this parameter, you define the set of words that you want to match and the word that will replace this match.

For example, with this definition, you are telling `ContextualParser` that when words `woman`, `female`, and `girl` are matched those will be replaced by `
female`, whereas `man`, `male`, `boy` and `gentleman` are matched those will be replaced by `male`.

female  woman   female  girl <br/>
male    man male    boy gentleman

So, for example for this text:

At birth, the typical boy is growing slightly faster than the typical girl, but the velocities become equal at about seven months, and then the girl grows faster until four years. From then until adolescence no differences in velocity can be detected.

The expected output of the annotator will be:

```val expectedResult = Array("boy", "girl", "girl")
val expectedMetadata =
Array(Map("field" -> "Gender", "normalized" -> "male", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"),
Map("field" -> "Gender", "normalized" -> "female", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"),
Map("field" -> "Gender", "normalized" -> "female", "confidenceValue" -> "0.13", "hits" -> "regex", "sentence" -> "0"))
```

For the `dictionary`, you just need to define a csv or tsv file, where the first element of the row is the normalized word, the other elements will be the values to match. You can define several words and elements to match just by adding another row and you set the path to the file on the parameter `setDictionary`.

The `dictionary` parameter is of the `typeExternalResource` by default the delimiter is `"\t"` you cand set another delimiter if you want according to your dictionary file format.

# Running a Pipeline

In [0]:
sample_text = """A 28 year old female with a history of gestational diabetes mellitus diagnosed eight years prior to 
presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis 
three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index 
( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting.
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . 
She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was 
significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , 
or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , 
anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin 
( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed 
as blood samples kept hemolyzing due to significant lipemia .
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior 
to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , 
the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , 
and lipase was 52 U/L .
 β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged 
 and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . 
 The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides 
 to 1400 mg/dL , within 24 hours .
 Twenty days ago.
 Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . 
 At birth the typical boy is growing slightly faster than the typical girl, but the velocities become equal at about 
 seven months, and then the girl grows faster until four years. 
 From then until adolescence no differences in velocity 
 can be detected. 21-02-2020 
21/04/2020
"""
data = spark.createDataFrame([[sample_text]]).toDF("text").cache()

data.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+
 text|
+----------------------------------------------------------------------------------------------------+
A 28 year old female with a history of gestational diabetes mellitus diagnosed eight years prior ...|
+----------------------------------------------------------------------------------------------------+

In [0]:
gender = '''male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,neutral'''

with open('/dbfs/gender.csv', 'w') as f:
    f.write(gender)


gender = {
  "entity": "Gender",
  "ruleScope": "sentence", 
  "completeMatchRegex": "true"
}

import json

with open('/dbfs/gender.json', 'w') as f:
    json.dump(gender, f)


date = {
  "entity": "Date ",
  "ruleScope": "sentence",
  "regex": "\\d{1,2}[\\/\\-\\:]{1}(\\d{1,2}[\\/\\-\\:]{1}){0,1}\\d{2,4}",
  "valuesDefinition":[],
  "prefix": [],
  "suffix": [],
  "contextLength": 150,
  "context": []
}

with open('/dbfs/date.json', 'w') as f:
    json.dump(date, f)


age = {
  "entity": "Age",
  "ruleScope": "sentence",
  "matchScope":"token",
  "regex" : "^[1][0-9][0-9]|[1-9][0-9]|[1-9]$",
  "prefix":["age of", "age"],
  "suffix": ["-years-old",
             "years-old",
             "-year-old",
             "-months-old",
             "-month-old",
             "-months-old",
             "-day-old",
             "-days-old",
             "month old",
             "days old",
             "year old",
             "years old", 
             "years",
             "year", 
             "months", 
             "old"
              ],
  "contextLength": 25,
  "context": [],
  "contextException": ["ago"],
  "exceptionDistance": 10
}

with open('/dbfs/age.json', 'w') as f:
    json.dump(age, f)

In [0]:
%fs ls dbfs:/

path,name,size
dbfs:/AntBNC_lemmas_ver_001.txt,AntBNC_lemmas_ver_001.txt,1348552
dbfs:/AskAPatient.fold-0.test.txt,AskAPatient.fold-0.test.txt,33834
dbfs:/AskAPatient.fold-0.train.txt,AskAPatient.fold-0.train.txt,594794
dbfs:/AskAPatient.fold-0.validation.txt,AskAPatient.fold-0.validation.txt,32162
dbfs:/FileStore/,FileStore/,0
dbfs:/JohnSnowLabs/,JohnSnowLabs/,0
dbfs:/MT_00.pdf,MT_00.pdf,528022
dbfs:/MT_01.pdf,MT_01.pdf,165397
dbfs:/MT_02.pdf,MT_02.pdf,309334
dbfs:/MT_03.pdf,MT_03.pdf,248334


In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")


In [0]:
gender_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_gender") \
        .setJsonPath("/dbfs/gender.json") \
        .setCaseSensitive(False) \
        .setContextMatch(False)\
        .setDictionary('file:/dbfs/gender.csv', read_as=ReadAs.TEXT, options={"delimiter":","})\
        .setPrefixAndSuffixMatch(False)        

In [0]:
age_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_age") \
        .setJsonPath("/dbfs/age.json") \
        .setCaseSensitive(False) \
        .setContextMatch(False)\
        .setPrefixAndSuffixMatch(False)

In [0]:
date_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_date") \
        .setJsonPath("/dbfs/date.json") \
        .setCaseSensitive(False) \
        .setContextMatch(False)\
        .setPrefixAndSuffixMatch(False)

In [0]:
chunkConverter_date = ChunkConverter().setInputCols("entity_date").setOutputCol("chunk_date")
chunkConverter_age = ChunkConverter().setInputCols("entity_age").setOutputCol("chunk_age")
chunkConverter_gender = ChunkConverter().setInputCols("entity_gender").setOutputCol("chunk_gender")
chunk_merger1 = ChunkMergeApproach().setInputCols(["chunk_date", "chunk_age"]).setOutputCol("merged_chunk1")
chunk_merger2 = ChunkMergeApproach().setInputCols(["merged_chunk1", "chunk_gender"]).setOutputCol("ner_chunk")

In [0]:
parserPipeline = Pipeline(stages=[
        document_assembler, 
        sentence_detector,
        tokenizer,
        gender_contextual_parser,
        age_contextual_parser,
        date_contextual_parser,
        chunkConverter_date,
        chunkConverter_age,
        chunkConverter_gender,
        chunk_merger1,
        chunk_merger2
        ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

parserModel = parserPipeline.fit(empty_data)

light_model = LightPipeline(parserModel)

In [0]:
annotations = light_model.fullAnnotate(sample_text)[0]
annotations.keys()

Out[14]: dict_keys(['chunk_age', 'document', 'ner_chunk', 'entity_gender', 'token', 'entity_date', 'merged_chunk1', 'entity_age', 'chunk_gender', 'chunk_date', 'sentence'])

## Highlight the Entities

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

ner_vis = visualiser.display(annotations, label_col='ner_chunk', document_col='document', return_html=True)
  
displayHTML(ner_vis)

A 28 Age year old female Gender with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting. Two weeks prior to presentation , she Gender was treated with a five-day course of amoxicillin for a respiratory tract infection . she Gender was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . she Gender had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her Gender abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she Gender reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her Gender glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Twenty days ago. her Gender euDKA was thought to be precipitated by her Gender respiratory tract infection in the setting of SGLT2 inhibitor use . At birth the typical boy Gender is growing slightly faster than the typical girl Gender , but the velocities become equal at about seven months, and then the girl Gender grows faster until four years. From then until adolescence no differences in velocity can be detected. 21-02-2020 Date 21/04/2020 Date